# Home Credit Default Risk by Kaggle

Inspired by the classic Kaggle problem: [Home Credit Default Risk](https://www.kaggle.com/competitions/home-credit-default-risk/overview), we are creating an ultimate cheatsheet for data scientist interviews. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold, cross_val_predict
import joblib
from sklearn import set_config
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from xgboost import XGBClassifier, plot_importance
from scipy.stats import loguniform, randint
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, roc_auc_score, f1_score, confusion_matrix, precision_recall_curve, classification_report
from outlier_and_nan_imputer import OutlierAndNaNImputer
import time
from utils import save_best_model, model_performance, check_missing_col_coverage

set_config(transform_output="pandas") 

In [ ]:
raw_data = pd.read_csv("data/application_train.csv")
previous_applications = pd.read_csv("etl/previous_applications_agg.csv")
bureau = pd.read_csv("etl/bureau_agg.csv")
bureau_balance = pd.read_csv("etl/bureau_bal_agg.csv")
pos_cash = pd.read_csv("etl/pos_cash_agg.csv")
installment_pay = pd.read_csv("etl/installment_pay.csv")
credit_card_agg = pd.read_csv("etl/credit_card_agg.csv")

In [ ]:

df = raw_data.copy()

df = df.merge(previous_applications, on="SK_ID_CURR", how="left", suffixes=["_main", "_previous_app"])
df = df.merge(bureau, on="SK_ID_CURR", how="left", suffixes=["_main", "_bureau"])
df = df.merge(pos_cash, on="SK_ID_CURR", how="left", suffixes=["_main", "_pos"])
df = df.merge(installment_pay, on="SK_ID_CURR", how="left", suffixes=["_main", "_repay"])
df = df.merge(bureau_balance, left_on="SK_ID_CURR", right_on="SK_ID_BUREAU", how="left", suffixes=["_main", "_bureau_bal"])
df = df.merge(credit_card_agg, on="SK_ID_CURR", how="left", suffixes=["_main", "_creditcard"])

df.head()

# 1. Check the target column

In [ ]:
# 1. Check target value
df.groupby("TARGET").size()

In [ ]:
df.drop(columns = ["TARGET", "SK_ID_CURR"], inplace=True)
trans_values = {}
col_transformers = []

This is a very **imbalanced data**, with label 1 less than 10% of label 0.
The way we usually handle imbalance data:

a) Data level:
Collect more data in minority class  
Undersample: might remove important data  
Oversample: might overfit  
To mitigate:   
Two-phase learning: train on resampled data, fine tune on original data  
Dynamic sampling: oversample the low-performing class and undersample the high performing class during training.  

b) Metric level: Check precision/recall, f1, AUC in ROC curve

c) Algorithm level: Add stronger penalty on misclassifying the minority class in loss function

# 2. Check predictors


2.0 Check the dtypes of predictors  
2.1 Check if all the numeric binary columns are in [0, 1]  
2.2 Check outliers in non-binary numeric cols, and impute them with null, medium or mean

In [ ]:
# 2. Check the predictors
df.dtypes.value_counts()

In [ ]:
check_missing_col_coverage(df)

In [ ]:
num_cols = df.select_dtypes(include='number').columns.tolist()
num_cols = [_ for _ in num_cols if _ not in ["SK_ID_CURR", "TARGET"]]
print(len(num_cols))

In [ ]:
num_cols_nonbinary = [x for x in num_cols if all(words not in x for words in ["FLAG", "NOT"])]
print(len(num_cols_nonbinary))
num_cols_binary = [_ for _ in num_cols if _ not in num_cols_nonbinary]
print(len(num_cols_binary))

In [ ]:
# 2.1 Check if all the numeric binary columns are either 0 or 1
invalid_cols = df[num_cols_binary].columns[~df[num_cols_binary].isin([0,1]).all()]
print(invalid_cols)
# Looks good!

In [ ]:
cat_cols = df.select_dtypes(include='object').columns.tolist()

Unlike Logistic Regression and linear regression models, xgboost handles missing values and outliers natively.

**Note!** In real-life ML practice, we always use the **training data only** for imputation or outlier detection. We can not foresee the stats of future unseen data in practice, and using test data would cause data leakage.  
In a Kaggle competition, however, we use train+test stats as it generally improves leaderboard score, but it won't generalize well outside the leaderboard.  
This notebook is for data scientist interview prep, so we always stick to real-life practice. The stats from training data are stored in a dictionary and will be applied to test data later.




# 3. Missing values in predictors

**Tree-based models**, like XGBoost, can handle missing values and no need to inpute.  
**Distance-based models**, linear Regression, SVM, KNN, Neural networks need imputation.



# 4. Feature Engineering

## 4.1 One-hot vector

In real pipeline, we use SimpleImputer to avoid dataleakage.

# Modeling

In [ ]:
# Prepare features

Y = raw_data["TARGET"]
X_cols = [_ for _ in df.columns if _ not in ["TARGET", "SK_ID_CURR"]]
X = df[X_cols]


In [ ]:
# Other column transformers


other_cat_cols_pipline = Pipeline([
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
]
)


col_transformers = [
    ("other_cat", other_cat_cols_pipline, cat_cols)
    
]

preprocessor = ColumnTransformer(transformers= col_transformers, remainder="passthrough" )#.set_output(transform="pandas") 


In [ ]:
X_cleaned = preprocessor.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_cleaned, Y, test_size=0.2, random_state=42, stratify=Y
)

# === 2) Class imbalance handling (on TRAIN only)
pos = int(np.sum(y_train == 1))
neg = int(np.sum(y_train == 0))
spw = neg / max(pos, 1)
print("scale_pos_weight:", round(spw, 3))

In [ ]:
# === 3) Hyperparameter search (AUC)
base = XGBClassifier(
    objective="binary:logistic",
    # max_depth=6,
    eval_metric="auc",
    # tree_method="hist",
    random_state=42,
    # learning_rate=0.1,      # fixed
    # subsample=0.8,          # fixed
    # colsample_bytree=0.8,   # fixed
    # min_child_weight=1,     # fixed
    # gamma=0.0,              # fixed
    # reg_alpha=0.0,          # fixed
    # reg_lambda=1.0,         # fixed
    use_label_encoder=False,
    scale_pos_weight=spw,
)


# --- Your tuned search space ---
param_dist = {
    "n_estimators": [300, 800],
    "max_depth": [4, 6, 8],
    "min_child_weight": [1, 3, 5],
    "gamma": [0, 0.3],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "reg_alpha": [0, 0.5],
    "reg_lambda": [1, 2],
    "learning_rate": [0.01, 0.05, 0.1],
}

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
search = RandomizedSearchCV(
    estimator=base,
    param_distributions=param_dist,
    n_iter=2,
    scoring="roc_auc",
    n_jobs=-1,
    cv=cv,
    verbose=1,
    random_state=42,
    refit=True     # refit on full TRAIN with best params 
)

search.fit(X_train, y_train)
print("Best AUC (CV):", search.best_score_)
print("Best params:", search.best_params_)

# Save the best model
save_best_model(search, "model/xgb_best_model.pkl")


In [ ]:
# Load model
loaded_model = joblib.load("model/xgb_best_model.pkl")
y_pred = loaded_model.predict(X_test)
y_pred_prob = loaded_model.predict_proba(X_test)[:, 1]

# Print model performance
model_performance(y_test, y_pred, y_pred_prob)

In [ ]:
plot_importance(loaded_model, max_num_features=15)
plt.show()

The roc-auc score is fine but f1-score is low. One possible reason is the default threshold 0.5 didn't cutoff properly.

# Submit

In [ ]:
test_data = pd.read_csv("data/application_test.csv")

test_data = test_data.merge(previous_applications, on="SK_ID_CURR", how="left", suffixes=["_main", "_previous_app"])
test_data = test_data.merge(bureau, on="SK_ID_CURR", how="left", suffixes=["_main", "_bureau"])
test_data = test_data.merge(pos_cash, on="SK_ID_CURR", how="left", suffixes=["_main", "_pos"])
test_data = test_data.merge(installment_pay, on="SK_ID_CURR", how="left", suffixes=["_main", "_repay"])
test_data = test_data.merge(bureau_balance, left_on="SK_ID_CURR", right_on="SK_ID_BUREAU", how="left", suffixes=["_main", "_bureau_bal"])
test_data = test_data.merge(credit_card_agg, on="SK_ID_CURR", how="left", suffixes=["_main", "_creditcard"])


In [ ]:
test_df = test_data[[_ for _ in test_data.columns if _!= "SK_ID_CURR"]]
test_df = preprocessor.transform(test_df)

In [ ]:
test_y_pred = loaded_model.predict(test_df)
test_y_pred_prob = loaded_model.predict_proba(test_df)[:, 1]


final = pd.DataFrame(test_data[["SK_ID_CURR"]])
final["TARGET"] = test_y_pred_prob

final.to_csv("submission_xgboost.csv", index=False, header=True)

This version got a 0.76 AUC score.